In [6]:
# import sys
# !conda install --yes --prefix {sys.prefix} requests

import requests
import pprint
import numpy as np

TMDB_API_KEY = "b468692645623d8d1a926f516015f1c1"
SEARCH_MOVIE_URL = "https://api.themoviedb.org/3/search/movie"
CREW_URL = "https://api.themoviedb.org/3/movie/{}/credits"
PERSON_MOVIE_CREDITS_URL = "https://api.themoviedb.org/3/person/{}/movie_credits"
GET_MOVIE_BY_ID_URL = "https://api.themoviedb.org/3/movie/{}"

def getParams():
    params = {
        'api_key':TMDB_API_KEY,
        'language': 'en-US'
    }
    return params

def searchForMovie(movName):
    queryParams = getParams()
    queryParams['query'] = movName
    response = requests.request("GET", SEARCH_MOVIE_URL, params=queryParams).json()
    movie = response["results"][0]
    return movie


def getActorsAndDirectors(movieId):
    queryParams = getParams()
    response = requests.request("GET", CREW_URL.format(movieId), params=queryParams).json()
    cast = response['cast']
    return cast

import time
def getActorTopMovies(actorId):
    queryParams = getParams()
    session = requests.session()
    response = session.request("GET", PERSON_MOVIE_CREDITS_URL.format(actorId), params=queryParams).json()
    cast = response["cast"]
    id_popularity_map = {}
    for movie in cast:
        id_popularity_map[movie['id']] = movie['popularity']
    from collections import OrderedDict
    sorted_popularity_dict = OrderedDict(sorted(id_popularity_map.items(), key=lambda x: x[1], reverse=True))
    movies_data_list = []
    s = time.time()
    for idx,movie_id in enumerate(sorted_popularity_dict):
        queryParams = getParams()
        response = session.request("GET", GET_MOVIE_BY_ID_URL.format(movie_id), params=queryParams).json()
        movies_data_list.append(response)
        if(idx == 10):
            break
    print("Time {}s".format(time.time() - s))
    return movies_data_list

'''
{
    "votes" : {
        "avg" : x,
        "min" : x,
        "max" : x
    },
    "revenue" : {
        "avg" : x,
        "min" : x,
        "max" : x
    },
    "budget" : {
        "avg" : x,
        "min" : x,
        "max" : x
    }
}
'''
def getMoviesStats(movie_list):
    votes, revenue, budget = [],[],[] 
    for movie in movie_list:
        if movie["vote_count"] < 50:
            pass
        if movie["vote_average"] != 0:
            votes.append(movie["vote_average"])
        if movie["revenue"] != 0:
            revenue.append(movie["revenue"])
        if movie["budget"] != 0:
            budget.append(movie["budget"])
    votes = np.array(votes)
    revenue = np.array(revenue)
    budget = np.array(budget)
    avg_vote = np.mean(votes)
    avg_revenue = np.mean(revenue)
    avg_budget = np.mean(budget)
    max_vote, max_revenue, max_budget = np.max(votes), np.max(revenue), np.max(budget)
    min_vote, min_revenue, min_budget = np.min(votes), np.min(revenue), np.min(budget)
    result = {
        "votes" : {
            "avg" : avg_vote,
            "min" : min_vote,
            "max" : max_vote
        },
        "revenue" : {
            "avg" : avg_revenue,
            "min" : min_revenue,
            "max" : max_revenue
        },
        "budget" : {
            "avg" : avg_budget,
            "min" : min_budget,
            "max" : max_budget
        }
    }
    return result

In [16]:
movie = searchForMovie("The Matrix")
crew = getActorsAndDirectors(movie["id"])
top_movies = getActorTopMovies(crew[0]["id"])

Time 0.5351381301879883s


In [17]:
stats = getMoviesStats(top_movies)

In [19]:
import pprint as pp

# pp.pprint(crew)
# pp.pprint(stats)
pp.pprint(top_movies)

[{'adult': False,
  'backdrop_path': '/umC04Cozevu8nn3JTDJ1pc7PVTn.jpg',
  'belongs_to_collection': {'backdrop_path': '/fSwYa5q2xRkBoOOjueLpkLf3N1m.jpg',
                            'id': 404609,
                            'name': 'John Wick Collection',
                            'poster_path': '/pZ4hR5IIoHHfNjot9rq4F96OGAi.jpg'},
  'budget': 20000000,
  'genres': [{'id': 28, 'name': 'Action'}, {'id': 53, 'name': 'Thriller'}],
  'homepage': 'http://www.johnwickthemovie.com/',
  'id': 245891,
  'imdb_id': 'tt2911666',
  'original_language': 'en',
  'original_title': 'John Wick',
  'overview': 'Ex-hitman John Wick comes out of retirement to track down the '
              'gangsters that took everything from him.',
  'popularity': 33.768,
  'poster_path': '/5vHssUeVe25bMrof1HyaPyWgaP.jpg',
  'production_companies': [{'id': 491,
                            'logo_path': '/rUp0lLKa1pr4UsPm8fgzmnNGxtq.png',
                            'name': 'Summit Entertainment',
                       